In [1]:
import tensorflow as tf
import numpy as np

In [2]:
train_file = open("../../data/iyer/train.txt")

In [3]:
train_file_contents = train_file.readlines()

In [4]:
len(train_file_contents)

52997

In [5]:
train = []
for line in train_file_contents:
    items = line.split('\t')
    if len(items) == 5:
        train.append(line.split('\t')[3])

In [6]:
len(train)

52795

In [7]:
len(train[0])

40

In [8]:
len(train[1])

294

In [9]:
train = [ex for ex in train if len(ex) < 80]

In [10]:
import sys
sys.path.append("../../src")
from text_data_utils import *
from os import path
import gensim

In [11]:
train_tok = tokenize_texts(train)

In [12]:
wv = gensim.models.Word2Vec(train_tok, size=300, min_count=3).wv

In [13]:
max_len = max(len(text) for text in train_tok)

In [14]:
train_tensor = tokenized_texts_to_tensor(train_tok, wv, max_len)

In [15]:
train_tensor[0, 0]

array([-0.1496356 ,  0.17530207, -0.06052994,  0.03097682, -0.24623041,
        0.24526589,  0.16078506, -0.14633822,  0.16734618, -0.03340849,
        0.18935007,  0.37714183,  0.3087871 ,  0.475375  ,  0.4593508 ,
       -0.08064191, -0.24868886, -0.03051438, -0.09899783,  0.21530603,
       -0.0971537 , -0.4988578 , -0.01944985, -0.32508117,  0.2467106 ,
       -0.11930186, -0.06406457, -0.00303773,  0.37440914, -0.19493656,
       -0.2417365 ,  0.16515574,  0.27625668, -0.20395383,  0.00207566,
        0.30956927,  0.13369949, -0.24815859, -0.04668345,  0.4591984 ,
       -0.09233286,  0.05893763,  0.12102162,  0.04708679, -0.14004228,
       -0.14284816,  0.00527399,  0.30506876,  0.389775  , -0.16201971,
        0.29084226,  0.03205778,  0.2913719 ,  0.00968005, -0.08877767,
        0.2222361 ,  0.12132713,  0.09659603, -0.07833605,  0.08442135,
       -0.38806307,  0.08455996, -0.00839686, -0.22626592, -0.07701427,
       -0.01066681, -0.07710035,  0.0969746 ,  0.01124953,  0.18

In [16]:
train_tensor_fl = np.reshape(train_tensor, (train_tensor.shape[0], train_tensor.shape[1] * train_tensor.shape[2]))

In [17]:
train_tensor.shape

(12921, 74, 300)

In [18]:
train_tensor_fl.shape

(12921, 22200)

In [19]:
val_file_contents = open('../../data/iyer/valid.txt').readlines()
val = []
for line in val_file_contents:
    items = line.split('\t')
    if len(items) == 5:
        sample = line.split('\t')[3]
        if len(sample) <= max_len:
            val.append(sample)
val_tok = tokenize_texts(val)
val_tensor = tokenized_texts_to_tensor(val_tok, wv, max_len)
val_tensor_fl = np.reshape(val_tensor, (val_tensor.shape[0], val_tensor.shape[1] * val_tensor.shape[2]))

In [20]:
from mlp_vae import MLPVariationalAutoEncoder

In [21]:
latent_dim = 256

In [22]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [23]:
wv.similar_by_vector(train_tensor[0,6])

[(';', 1.0000001192092896),
 ('true', 0.9312881231307983),
 ('false', 0.9302204847335815),
 ('value', 0.9240065813064575),
 ('Editable', 0.9211550951004028),
 ('OR', 0.9210493564605713),
 ('10', 0.9207760095596313),
 ('bit', 0.913860559463501),
 ('scope', 0.9114536046981812),
 ('100', 0.901944637298584)]

In [24]:
model = MLPVariationalAutoEncoder(train_tensor_fl.shape[1], latent_dim, final_activation=None)
model.train(train_tensor_fl, val_tensor_fl, 3, 512, tf.keras.optimizers.Adam(learning_rate=0.0001))

KeyboardInterrupt: 

In [ ]:
import random

In [ ]:
random.seed()
random_idx = random.randrange(train_tensor.shape[0])

In [ ]:
print("(Training Set) Input: ", tensor_to_tokenized_texts(np.array([train_tensor[random_idx]]), wv)[0])

In [ ]:
rec = np.reshape(model.decode(model.encode(np.array([train_tensor_fl[random_idx]])).sample()), (1, train_tensor.shape[1], train_tensor.shape[2]))

In [ ]:
print("(Training Set) Reconstructed: ", tensor_to_tokenized_texts(rec, wv)[0])

In [ ]:
wv.similar_by_vector(train_tensor[random_idx, 3])

In [ ]:
wv.similar_by_vector(rec[0, 3])

In [ ]:
test_file_contents = open('../../data/iyer/test.txt').readlines()
test = []
for line in test_file_contents:
    items = line.split('\t')
    if len(items) == 5:
        sample = line.split('\t')[3]
        if len(sample) < max_len:
            test.append(sample)
test_tok = tokenize_texts(test)
test_tensor = tokenized_texts_to_tensor(test_tok, wv, max_len)
test_tensor_fl = np.reshape(test_tensor, (test_tensor.shape[0], test_tensor.shape[1] * test_tensor.shape[2]))

In [ ]:
random.seed()
random_idx = random.randrange(test_tensor.shape[0])

In [ ]:
print("(Test Set) Input: ", tensor_to_tokenized_texts(np.array([test_tensor[random_idx]]), wv)[0])

In [ ]:
print("(Test Set) Reconstructed: ", tensor_to_tokenized_texts(np.reshape(model.decode(model.encode(np.array([test_tensor_fl[random_idx]])).sample()), (1, test_tensor.shape[1], test_tensor.shape[2])), wv)[0])